<a href="https://colab.research.google.com/github/kashishthakur26/Deblur-image-app/blob/main/Deblur_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
from shutil import copyfile

import click
import tqdm

def reorganize_gopro_files(dir_in, dir_out):
    if not os.path.exists(dir_out):
        os.makedirs(dir_out)

    for folder_train_test in tqdm.tqdm(os.listdir(dir_in), desc='dir'):
        output_directory = os.path.join(dir_out, folder_train_test)
        output_directory_A = os.path.join(output_directory, 'A')
        output_directory_B = os.path.join(output_directory, 'B')
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        if not os.path.exists(output_directory_A):
            os.makedirs(output_directory_A)
        if not os.path.exists(output_directory_B):
            os.makedirs(output_directory_B)

        current_folder_path = os.path.join(dir_in, folder_train_test)
        for image_folder in tqdm.tqdm(os.listdir(current_folder_path), desc='image_folders'):

            current_sub_folder_path = os.path.join(current_folder_path, image_folder)

            for image_blurred in os.listdir(os.path.join(current_sub_folder_path, 'blur')):
                current_image_blurred_path = os.path.join(current_sub_folder_path, 'blur', image_blurred)
                output_image_blurred_path = os.path.join(output_directory_A, image_folder + "_" + image_blurred)
                copyfile(current_image_blurred_path, output_image_blurred_path)

            for image_sharp in os.listdir(os.path.join(current_sub_folder_path, 'sharp')):
                current_image_sharp_path = os.path.join(current_sub_folder_path, 'sharp', image_sharp)
                output_image_sharp_path = os.path.join(output_directory_B, image_folder + "_" + image_sharp)
                copyfile(current_image_sharp_path, output_image_sharp_path)



In [1]:
import os
from shutil import copyfile
import tqdm

In [2]:
def reorganize_gopro_files(dir_in, dir_out):
  if not os.path.exists(dir_out):
        os.makedirs(dir_out)

  for folder_train_test in tqdm.tqdm(os.listdir(dir_in), desc='dir'):
        output_directory = os.path.join(dir_out, folder_train_test)
        output_directory_A = os.path.join(output_directory, 'A')
        output_directory_B = os.path.join(output_directory, 'B')
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        if not os.path.exists(output_directory_A):
            os.makedirs(output_directory_A)
        if not os.path.exists(output_directory_B):
            os.makedirs(output_directory_B)

        current_folder_path = os.path.join(dir_in, folder_train_test)
        for image_folder in tqdm.tqdm(os.listdir(current_folder_path), desc='image_folders'):

            current_sub_folder_path = os.path.join(current_folder_path, image_folder)

            for image_blurred in os.listdir(os.path.join(current_sub_folder_path, 'blur')):
                current_image_blurred_path = os.path.join(current_sub_folder_path, 'blur', image_blurred)
                output_image_blurred_path = os.path.join(output_directory_A, image_folder + "_" + image_blurred)
                copyfile(current_image_blurred_path, output_image_blurred_path)

            for image_sharp in os.listdir(os.path.join(current_sub_folder_path, 'sharp')):
                current_image_sharp_path = os.path.join(current_sub_folder_path, 'sharp', image_sharp)
                output_image_sharp_path = os.path.join(output_directory_B, image_folder + "_" + image_sharp)
                copyfile(current_image_sharp_path, output_image_sharp_path)



In [3]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf


RESHAPE = (256,256)

def is_an_image_file(filename):
    IMAGE_EXTENSIONS = ['.png', '.jpg', '.jpeg']
    for ext in IMAGE_EXTENSIONS:
        if ext in filename:
            return True
    return False


def list_image_files(directory):
    files = sorted(os.listdir(directory))
    return [os.path.join(directory, f) for f in files if is_an_image_file(f)]


def load_image(path):
    img = Image.open(path)
    return img


def preprocess_image(cv_img):
    cv_img = cv_img.resize(RESHAPE)
    img = np.array(cv_img)
    img = (img - 127.5) / 127.5
    return img


def deprocess_image(img):
    img = img * 127.5 + 127.5
    return img.astype('uint8')


def save_image(np_arr, path):
    img = np_arr * 127.5 + 127.5
    im = Image.fromarray(img)
    im.save(path)


def load_images(path, n_images):
    if n_images < 0:
        n_images = float("inf")
    A_paths, B_paths = os.path.join(path, 'A'), os.path.join(path, 'B')
    all_A_paths, all_B_paths = list_image_files(A_paths), list_image_files(B_paths)
    images_A, images_B = [], []
    images_A_paths, images_B_paths = [], []
    for path_A, path_B in zip(all_A_paths, all_B_paths):
        img_A, img_B = load_image(path_A), load_image(path_B)
        images_A.append(preprocess_image(img_A))
        images_B.append(preprocess_image(img_B))
        images_A_paths.append(path_A)
        images_B_paths.append(path_B)
        if len(images_A) > n_images - 1: break

    return {
        'A': np.array(images_A),
        'A_paths': np.array(images_A_paths),
        'B': np.array(images_B),
        'B_paths': np.array(images_B_paths)
    }

def write_log(callback, names, logs, batch_no):
    """
    Util to write callback for Keras training
    """
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()


In [4]:
!pip install keras.utils

  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2631 sha256=6a2b486891b784fe5fdb2496a98fa681605b726f8ff6632fdaf43dc8ec2a85d9
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputSpec, Layer,Add, Input, Conv2D, Activation, BatchNormalization, Dropout
from tensorflow.python.keras.utils import conv_utils
from tensorflow.keras.backend import image_data_format


In [3]:
def res_block(input, filters, kernel_size=(3,3), strides=(1,1), use_dropout=False):
    x = ReflectionPadding2D((1,1))(input)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
      x = Dropout(0.5)(x)

    x = ReflectionPadding2D((1,1))(x)
    x = conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)

    merged = Add()([input, x])
    return merged

In [8]:
def spatial_reflection_2d_padding(x, padding=((1,1), (1,1)), data_format=None):
  assert len(padding)==2
  assert len(padding[0]) ==2
  assert len(padding[1]) == 2
  if data_format is None:
    data_format = image_data_format()
  if data_format not in {'channels_first', 'channels_last'}:
    raise ValueError('Unknown data_format' + str(data_format))

  if data_format == 'channels_first':
    pattern = [[0,0],
               [0,0],
               list(padding[0]),
               list(padding[1])]
  else:
    pattern = [[0,0],
               list(padding[0]),
               list(padding[1]),
               [0,0]]
  return tf.pad(x, pattern, "REFLECT")


In [6]:
class ReflectionPadding2D(Layer):
  def  __init__(self,
                padding=(1,1),
                data_format=None,
                **kwargs):
    super(ReflectionPadding2D, self).__init__(**kwargs)
    self.data_format = conv_utils.normalize_data_format(data_format)
    if isinstance(padding, int):
      self.padding = ((padding, padding), (padding, padding))
    elif hasattr(padding, '__len__'):
      if len(padding) != 2:
        raise ValueError('`padding` should have two elements. '
                         'Found: '+ str(padding))

      height_padding = conv_utils.normalize_tuple(padding[0], 2,
                                                  '1st entry of padding')
      width_padding = conv_utils.normalize_tuple(padding[1], 2,
                                                       '2nd entry of padding')
      self.padding = (height_padding, width_padding)
    else:
        raise ValueError('`padding` should be either an int, '
                             'a tuple of 2 ints '
                             '(symmetric_height_pad, symmetric_width_pad), '
                             'or a tuple of 2 tuples of 2 ints '
                             '((top_pad, bottom_pad), (left_pad, right_pad)). '
                             'Found: ' + str(padding))
    self.input_spec = InputSpec(ndim=4)

  def compute_output_shape(self, input_shape):
    if self.data_format == 'channels_first':
      if input_shape[2] is not None:
        rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
      else:
          rows = None
      if input_shape[3] is not None:
          cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
      else:
          cols = None
      return (input_shape[0],
              input_shape[1],
              rows,
              cols)
    elif self.data_format == 'channels_last':
        if input_shape[1] is not None:
            rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
        else:
            rows = None
        if input_shape[2] is not None:
            cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
        else:
            cols = None
        return (input_shape[0],
                rows,
                cols,
                input_shape[3])

  def call(self, inputs):
    return spatial_reflection_2d_padding(inputs,
                                         padding=self.padding,
                                         dat_format= self.data_format)
  def get_config(self):
    config = {'padding':self.padding,
              'data_format': self.data_format}
    base_config = super(ReflectionPadding2D, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))



In [10]:
from keras.layers import Input, Activation, Add, UpSampling2D, LeakyReLU, Conv2D, Dense, Flatten, Lambda, BatchNormalization
from keras.models import Model

In [1]:
channel_rate = 64
image_shape = (256,256,3)
patch_shape = (channel_rate, channel_rate, 3)

ngf = 64
ndf = 64
input_nc = 3
output_nc =3
input_shape_generator = (256,256,input_nc)
input_shape_discriminator = (256,256,output_nc)

n_blocks_gen = 9


def generator_model():

  inputs = Input(shape=image_shape)
  x = ReflectionPadding2D((3,3))(inputs)
  x = Conv2D(filters=ngf, kernel_size=(7,7), padding='valid')
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  n_downsampling = 2

  for i in range(n_downsampling):
    mult = 2**i
    x = Conv2D(filters=ngf*mult*2, kernel_size=(3,3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

  mult = 2**n_downsampling
  for i in range(n_blocks_gen):
    x = res_block(x, ngf*mult, use_dropout=True)

  for i in range(n_downsampling):
    mult = 2**(n_downsampling)
    x = UpSampling2D()(x)
    x = Conv2D(filters=int(ngf*mult/2), kernel_size=(3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

  x = ReflectionPadding2D((3,3))(x)
  x = Conv2D(filters=output_nc, kernel_size=(7,7), padding='valid')(x)
  x = Activation('tanh')(x)

  outputs = Add()([x, inputs])
  outputs = Lambda(lambda z: z/2)(outputs)

  model = Model(inputs=inputs, outputs=outputs, name='Generator')
  return model

In [2]:
def discriminator_model():
  n_layers, use_sigmoid =3, False
  inputs = Input(shape=input_shape_discriminator)

  x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
  x = LeakyReLU(0.2)(x)

  nf_mult, nf_mult_prev = 1, 1
  for n in range(n_layers):
    nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

  nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
  x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
  x = BatchNormalization()(x)
  x = LeakyReLU(0.2)(x)

  x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
  if use_sigmoid:
    x =Activation('sigmoid')(x)

  x = Flatten()(x)
  x = Dense(1024, activation='tanh')(x)
  x = Dense(1, activation = 'sigmoid')(x)

  model = Model(inputs=inputs, outputs=x, name='Discriminator')
  return model


In [3]:
def generator_containing_discriminator(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [4]:
def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_image = generator(inputs)
    outputs = discriminator(generated_image)
    model = Model(inputs=inputs, outputs=[generated_image, outputs])
    return model

In [5]:
import tensorflow.keras.backend as K
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import numpy as np


In [6]:
image_shape = (256,256,3)

def l1_loss(y_true, y_pred):
  return K.mean(K.abs(y_pred - y_true))

def perceptual_loss_100(y_true, y_pred):
  return 100*perceptual_loss(y_true, y_pred)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))


def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)


def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))

    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)

    return K.mean(gradient_penalty)

In [9]:
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import datetime

In [10]:
BASE_DIR = 'weigths/'

def save_all_weights(d, g, epoch_number, current_loss):
  now = datetime.datetime.now()
  save_dir = os.path.join(BASE_DIR, '{}{}'.format(now))
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
  d.save_weights(os.path.join(save_dir, 'discriminator_{}_{}.h5'.format(epoch_number)), True)


In [12]:
def train_multiple_outputs(n_images, batch_size, epoch_num, critic_updates=5):
  data = load_images('./images/train', n_images)
  y_train, x_train = data['B'], data['A']

  g = generator_model()
  d = discriminator_model()

  d_on_g = generator_containing_discriminator_multiple_outputs(g,d)

  d_opt = Adam(lr=1e-4, beta_1=0.9, beta_2= 0.999, epsilon=1e-08)
  d_on_g_opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

  d.trainable = True
  d.compile(optimizer=d_opt, loss=wasserstein_loss)
  d.trainable = False
  loss = [perceptual_loss, wasserstein_loss]
  loss_weights = [100,1]
  d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
  d.trainable = True
  output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))
  log_path = './logs'
  tensorboard_callback = TensorBoard(log_path)

  for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        with open('log.txt', 'a+') as f:
            f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))

